<a href="https://colab.research.google.com/github/aletscn/NLP-REAL-OR-NOT-/blob/master/cleanTweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import sys

import pandas as pd
import numpy as np
from collections import defaultdict
import nltk
from nltk.corpus import stopwords
stop=set(stopwords.words('english'))
from nltk.tokenize import word_tokenize
import string
import re
from PIL import Image    # to import the image


%matplotlib inline

#para suprimir notacion cientifica en los outputs
pd.options.display.float_format='{:20,.1f}'.format

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
#Donde se encuentran los documentos (train, text)
DRIVE_DIR='/content/drive'
DATA_DIR = 'drive/My Drive/Colab Notebooks'
TWEETS_DIR='data_tp2'
#GLOVE_DIR = os.path.join(BASE_DIR, 'glove.6B')

In [4]:
from google.colab import drive
drive.mount(DRIVE_DIR)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
os.chdir(DATA_DIR)

In [6]:
ls

 20_newsgroup/                              glove.6B/
'22 5.ipynb'                                news20.tar.gz
 cleanTweets.ipynb                         'TP1 version Ale v2.ipynb'
'Copy of Ejercicio Clase 20-4-2020.ipynb'   twitter.png
 data_tp2/                                  Untitled0.ipynb


In [7]:
#Loading CSVs

train=pd.read_csv(os.path.join(TWEETS_DIR, 'train.csv'))
test=pd.read_csv(os.path.join(TWEETS_DIR, 'test.csv'))

#Filling NAs
train['keyword']=train['keyword'].fillna('no_keyword')
train['location']=train['location'].fillna('no_location')

test['keyword']=test['keyword'].fillna('no_keyword')
test['location']=test['location'].fillna('no_location')

#Filtering duplicated tweets, deleting contradictory labeled tweets (ONLY FOR TRAIN DF)

train_clean=train
duplicate_texts=train_clean.loc[:,['text','target']].groupby(['text']).transform('count')
train_clean['duplicate_text_count']=duplicate_texts

contradiction=train_clean.loc[:,['text','target']].groupby(['text']).transform('nunique')
train_clean['count_targets']=contradiction
train_clean=train_clean[train.count_targets==1]
train_clean.drop('count_targets',axis=1,inplace=True)

train_clean.drop_duplicates(subset ='text', keep = 'first', inplace = True)
print('El DataFrame post cleaning cuenta con {} filas y {} columnas'.format(train_clean.shape[0],train_clean.shape[1]))


El DataFrame post cleaning cuenta con 7485 filas y 6 columnas


In [8]:
#Cleaning URLs, punctuations, special characters


def deleteSpecials(inputString):
  text = re.sub(r'http\S+', '', inputString)  
  text2 = re.sub(r'@\S+', '', text)
  text3 = re.sub(r'&\S+', '', text2)
  text4 = re.sub(r'Ã\S+', '', text3)
  text5 = re.sub(r'www\S+','',text4)
  text6 = re.sub(r'\x89Û\S+','',text4)
  return (text6)

def decontracted(inputString):
    phrase = re.sub(r"won't", "will not", inputString)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"couldn", "could", phrase)
    phrase = re.sub(r"don", "do", phrase)
    phrase = re.sub(r"doesn", "does", phrase)
    phrase = re.sub(r"isn", "is", phrase)
    phrase = re.sub(r"mustn", "must", phrase)
    phrase = re.sub(r"shouldn", "should", phrase)
    phrase = re.sub(r"wasn", "was", phrase)

    # general

    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    
    return phrase

def deletePunctuation(inputString):
  aux=string.punctuation
  special_characters='ÃŒ©‰¥¼ªû'
  #puncts=aux.translate(str.maketrans("#@:/.","     "))
  auxstring=" " * len(aux)
  auxstring2=" " * len(special_characters)
  outputString=inputString.translate(str.maketrans(aux,auxstring))
  outputString=outputString.translate(str.maketrans(special_characters,auxstring2))
  return outputString

def deleteDigits(value):
    blist2 = [item for item in value if not item.isdigit()]
    blist3 = "".join(blist2)
    return blist3

def cleanText(inputString):
  text_1=deleteSpecials(inputString)
  text_2=decontracted(text_1)
  text_3=deletePunctuation(text_2)
  text_4=deleteDigits(text_3)
  return text_4



train_clean['text_clean'] = train_clean['text'].apply(lambda x: cleanText(x))
test_clean=test
test_clean['text_clean'] = test_clean['text'].apply(lambda x: cleanText(x))



In [9]:
train_clean['duplicate_text_clean_count']=train_clean.loc[:,['text_clean','target']].groupby(['text_clean']).transform('count')
train_clean['count_targets']=train_clean.loc[:,['text_clean','target']].groupby(['text_clean']).transform('nunique')
train_clean=train_clean[train_clean.count_targets==1]
train_clean.drop('count_targets',axis=1,inplace=True)
train_clean.drop_duplicates(subset ='text_clean', keep = 'first', inplace = True)
print('El DataFrame pre cleaning cuenta con {} filas y {} columnas'.format(train.shape[0],train.shape[1]))
print('El DataFrame post cleaning cuenta con {} filas y {} columnas'.format(train_clean.shape[0],train_clean.shape[1]))

train_clean.drop('duplicate_text_count',axis=1,inplace=True)
train_clean.drop('duplicate_text_clean_count',axis=1,inplace=True)


El DataFrame pre cleaning cuenta con 7613 filas y 7 columnas
El DataFrame post cleaning cuenta con 6855 filas y 8 columnas


In [10]:
def deleteStopwords(inputString):
  inputString= inputString.lower()
  text_tokens = word_tokenize(inputString)
  tokens_without_sw = [word for word in text_tokens if not word in stopwords.words('english')]
  filtered_sentence = (" ").join(tokens_without_sw)
  return filtered_sentence

train_clean['text_clean_nosw'] = train_clean['text_clean'].apply(lambda x:deleteStopwords(x))
test_clean['text_clean_nosw'] = test_clean['text_clean'].apply(lambda x:deleteStopwords(x))


In [12]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

def sentenceLem(sentence):
    wordnet_lemmatizer = WordNetLemmatizer()
    sentence_words = nltk.word_tokenize(sentence)
    temp_line=[] 
    lines_with_lemmas=[]
    
    for word in sentence_words:
        temp_line.append(wordnet_lemmatizer.lemmatize(word, pos="v"))

    string=' ' 
    lines_with_lemmas.append(string.join(temp_line)) 
    lines=lines_with_lemmas
    str1=''
    lines=str1.join(lines)
    return lines

train_clean['text_clean_lem'] = train_clean['text_clean_nosw'].apply(lambda x:sentenceLem(x))
test_clean['text_clean_lem'] = test_clean['text_clean_nosw'].apply(lambda x:sentenceLem(x))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [13]:
from nltk.tokenize import word_tokenize

tokenized_sents = [word_tokenize(i) for i in train_clean['text_clean_lem']]
train_clean['lem_tokenized']=tokenized_sents

tokenized_sents = [word_tokenize(i) for i in test_clean['text_clean_lem']]
test_clean['lem_tokenized']=tokenized_sents

In [16]:
train_clean.to_csv("train_clean.csv", index=False, header=True)

test_clean.to_csv("test_clean.csv", index=False, header=True)


In [17]:
ls

 20_newsgroup/                              news20.tar.gz
'22 5.ipynb'                                test_clean.csv
 cleanTweets.ipynb                         'TP1 version Ale v2.ipynb'
'Copy of Ejercicio Clase 20-4-2020.ipynb'   train_clean.csv
 data_tp2/                                  twitter.png
 glove.6B/                                  Untitled0.ipynb
